# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
cityCsv = "city_weather.csv"
cityWeatherDF = pd.read_csv("city_weather.csv")
cityWeatherDF.head()

,Unnamed: 0,CityID,CityName,Country,Lat,Lng,Temperature,MaxTemp,Humidity,Cloudiness,WindSpeed
0,0,2155415,New Norfolk,AU,-42.78,147.06,80.64,84.99,28,100,19.46
1,1,5106841,Albany,US,42.60,-73.97,16.38,21.20,67,90,3.36
2,2,3407407,Aquiraz,BR,-3.90,-38.39,82.26,82.40,78,40,6.93
3,3,2015306,Tiksi,RU,71.69,128.87,-26.07,-26.07,89,85,4.56
4,4,3871336,Santiago,CL,-33.46,-70.65,68.16,70.00,59,0,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:

locations = cityWeatherDF[["Lat", "Lng"]]
humidity = cityWeatherDF["Humidity"].astype(float)
humidityMax = humidity.max()
print(humitityMax)
fig = gmaps.figure()

heatLayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius=3)
fig.add_layer(heatLayer)
fig

100.0


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
maxTemperatureFilterDF = cityWeatherDF[(cityWeatherDF['MaxTemp'] < 80.0) & (cityWeatherDF['MaxTemp'] > 70.0)]
maxTempHumidityFilterDF = maxTemperatureFilterDF[(maxTemperatureFilterDF['Humidity']<60) & (maxTemperatureFilterDF['Humidity']>20)]
perfectVacationDF = maxTempHumidityFilterDF[(maxTempHumidityFilterDF['Cloudiness'] < 50)].dropna()
perfectVacationDF

,Unnamed: 0,CityID,CityName,Country,Lat,Lng,Temperature,MaxTemp,Humidity,Cloudiness,WindSpeed
123,123,3837056,San Luis,AR,-33.30,-66.34,78.87,78.87,47,3,11.25
147,147,933692,Kang,BW,-23.68,22.79,78.91,78.91,45,17,12.33
195,195,1259395,Porbandar,IN,21.64,69.61,75.20,75.20,31,0,10.29
229,229,2070998,Geraldton,AU,-28.77,114.60,78.80,78.80,44,0,25.28
263,263,1257060,Satana,IN,20.58,74.20,78.04,78.04,40,12,8.99
330,330,1254046,Tura,IN,25.52,90.22,73.24,73.24,41,0,4.56
336,336,1309937,Maymyo,MM,22.03,96.47,75.20,75.20,53,0,3.36
383,383,933366,Maun,BW,-19.98,23.42,76.06,76.06,59,9,5.59
391,391,1263752,Mangrol,IN,21.12,70.12,72.75,72.75,46,0,8.25
414,414,2367568,Bassar,TG,9.25,0.78,70.70,70.70,38,36,2.86


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotelDF = pd.DataFrame(columns=['Hotel Name', 'CityID'])

for index,row in perfectVacationDF.iterrows():
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_search = "Hotels"
    target_radius = 5000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response = response.json()
    if response['status'] =='OK':
        hotelDF = hotelDF.append({'Hotel Name': response["results"][0]["name"],
                        'CityID': row['CityID']}, ignore_index=True)
    else:
        print('No hotels for requested City')

hotelWeatherMergeDF = pd.merge(hotelDF,perfectVacationDF, on='CityID')
hotelWeatherMergeDF = hotelWeatherMergeDF[['CityID', 'CityName', 'Country', 'Lat', 'Lng', 'Hotel Name', 'Temperature', 'MaxTemp', 'Humidity', 'Cloudiness', 'WindSpeed']]
hotelWeatherMergeDF.set_index('CityID', inplace=True)
hotelWeatherMergeDF

No hotels for requested City
No hotels for requested City


,CityName,Country,Lat,Lng,Hotel Name,Temperature,MaxTemp,Humidity,Cloudiness,WindSpeed
CityID,,,,,,,,,,
3837056,San Luis,AR,-33.30,-66.34,Vista Suites Spa & Golf,78.87,78.87,47,3,11.25
933692,Kang,BW,-23.68,22.79,Nkisi Guesthouse,78.91,78.91,45,17,12.33
1259395,Porbandar,IN,21.64,69.61,Lords Inn Porbandar,75.20,75.20,31,0,10.29
2070998,Geraldton,AU,-28.77,114.60,Ocean Centre Hotel,78.80,78.80,44,0,25.28
1257060,Satana,IN,20.58,74.20,Shabari hotel Satana,78.04,78.04,40,12,8.99
1254046,Tura,IN,25.52,90.22,Hotel RIKMAN Continental,73.24,73.24,41,0,4.56
1309937,Maymyo,MM,22.03,96.47,Aureum Palace Hotel & Resort Pyin Oo Lwin,75.20,75.20,53,0,3.36
933366,Maun,BW,-19.98,23.42,Cresta Riley's,76.06,76.06,59,9,5.59


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{CityName}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelWeatherMergeDF.iterrows()]
locations = hotelWeatherMergeDF[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display Map
fig.add_layer(markerLayer)
fig

Figure(layout=FigureLayout(height='420px'))